In [2]:
def getPrimoral(k):
    r = 1
    p = 2
    for i in range(k):
        while not is_prime(p):
            p += 1
        r *= p
        p += 1
    return r

In [3]:
M = getPrimoral(39)

In [4]:
ordM = mod(65537, M).multiplicative_order()

In [5]:
256 + float(log(ordM / M) / log(2))

97.89712030683688

In [3]:
import telnetlib
import json

host = "socket.cryptohack.org"
port = int(13386)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

def get_data():
    opening = readline()
    json_send({
        "option" : "get_flag"
    })
    data = json_recv()
    return data

In [73]:
def trimZero(f):
    while len(f) > 0 and f[-1] == 0:
        f.pop()
    return f

def fillZero(f, lenFill):
    assert len(f) <= lenFill
    return f + [0] * (lenFill - len(f))
        
def modPoly(f, modulus):
    r = f.copy()
    for i in range(len(f)):
        r[i] %= modulus
    return trimZero(r)

def makeMonic(f, modulus):
    r = f.copy()
    leadCoefficientInverse = inverse_mod(r[-1], modulus)
    for i in range(len(r)):
        r[i] *= leadCoefficientInverse
        r[i] %= modulus
    return r

def addPoly(f1, f2):
    f3 = [0] * max(len(f1), len(f2))
    for i in range(len(f3)):
        if i < min(len(f1), len(f2)):
            f3[i] = f1[i] + f2[i]
        elif len(f1) < len(f2):
            f3[i] = f2[i]
        else:
            f3[i] = f1[i]

def mulPoly(f1, f2):
    f3 = [0] * (len(f1) + len(f2))
    for i2 in range(len(f2)):
        for i1 in range(len(f1)):
            f3[i1 + i2] += f1[i1] * f2[i2]
    return trimZero(f3)

def powPoly(f, p):
    r = [1]
    for i in range(p):
        r = mulPoly(r, f)
    return r

def getRoots(f):
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    fInPolyRep = 0
    for i in range(len(arr)):
        fInPolyRep += f[i] * _x_ ^ i
    return fInPolyRep.roots()

def coppersmithAttack(f, modulus, m, X=-1):
    # Assure monic
    assert f[-1] == 1
    
    # Get degree & set X if necessary
    degree = len(f) - 1
    assert degree >= 1
    if X == -1:
        X = int(modulus ^ (1 / degree))
        print('[No upperbound is set so the algorithm set it to {}]'.format(X))
    
    # Get dim
    dimension = (degree) * (m + 1)

    # Constructs a matrix.
    M = []
    for v in range( 0, m+1 ):
        for u in range( 0, degree ):
            # Generate g(x) function from f(x)
            g = mulPoly( mulPoly( [modulus^(m-v)], [0]*u+[1] ), powPoly(f, v) )
            
            # Create g(xX) from g(x)
            for i in range(len(g)):
                g[i] *= X^i
            
            # Append to row
            M.append( fillZero(g, dimension) )
    M = matrix(M)
    
    # Assert if our matrix meets the criteria
    det = 1
    for i in range(dimension):
        det *= M[i, i]
    assert int((det ^ (1/dimension)) * (2 ^ (dimension/4))) < int(modulus ^ m / isqrt(dimension))
    
    # Apply LLL
    M.LLL()
    
    # Get the shortest vector, return as the new polynomial
    r = trimZero(M[0])
    for i in range(len(r)):
        r[i] *= inverse_mod(X^i, modulus)
        r[i] %= modulus
    return getRoots(r)

In [74]:
def hastad(fs, Ns):
    # Setup variables
    Ts    = [1] * len(fs)
    ProdN = 1
    for i in range(len(fs)):
        ProdN *= Ns[i]
        
    # Generate Ti, Chinese Remainder Coefficients
    for i in range(len(fs)):
        Ts[i] *= ProdN / Ns[i]
        Ts[i] *= inverse_mod(Ts[i], Ns[i])
        Ts[i] %= ProdN
        
    # Generate f(x) from fi(x)s
    f = [0]
    for i in range(len(fs)):
        f = addPoly(f, mulPoly([Ts[i]], fs[i]))
    f = modPoly(f, ProdN)
    
    # Make sure we actually had monic polynomial
    assert f[-1] == 1

In [72]:
fs = []
Ns = []
for _ in range(12):
    tn = telnetlib.Telnet(host, port)
    data = get_data()

    c    = int(data["encrypted_flag"])
    N    = int(data["modulus"])
    a, b = int(data["padding"][0]), int(data["padding"][1])

    f = modPoly(powPoly([b, a], 11), N)
    f[0] -= c
    f = modPoly(f, N)
    
    fs.append(makeMonic(f, N))
    Ns.append(N)

In [1]:
hastad(fs, Ns)

NameError: name 'hastad' is not defined